In [1]:
import pandas as pd
import numpy as np

In [41]:
user = pd.read_csv('./whyout_data/user.csv') # shape(31178,13), user idx에서 2354개가 비어있음
place = pd.read_csv('./whyout_data/place.csv') # shape(4697,10), place idx에서 23개가 비어있음
product = pd.read_csv('./whyout_data/product.csv') # shape(5834,11), product idx에서 538개가 비어있음
video = pd.read_csv('./whyout_data/video.csv') # shape(3250, 9), video idx에서 315개가 비어있음
user_place = pd.read_csv('./whyout_data/user_place.csv') # shape (31176,4697) 유저 x 장소 아이템 # 8756명이 장소를 평가하지 않음
user_product = pd.read_csv('./whyout_data/user_product.csv') # shape(31176,5834) 유저 x 상품 아이템
user_video = pd.read_csv('./whyout_data/user_video.csv') # shape (31177, 3250) 유저 x 영상 아이템

In [6]:
# # idx missing value 찾기
# df = video.sort_values('idx')
# # 연속된 숫자 생성 (전체 범위의 시작과 끝을 기반으로)
# full_range = pd.Series(range(df['idx'].min(), df['idx'].max() + 1))
# # 누락된 숫자 찾기
# missing_numbers = full_range[~full_range.isin(df['idx'])]
# print(len(missing_numbers))

In [7]:
# # NaN 값 찾기
# nan_indices = np.where(place['주변환경'].isna())[0]
# count = 0

# if len(nan_indices) > 0:
#     print(f'총 행 수: {len(place)}, NaN 행 수: {len(nan_indices)}')
#     print("해당 열에서 NaN 값이 발견된 위치는 다음과 같습니다:")
#     for idx in nan_indices:
#         print(f"인덱스: {idx}, 값: NaN")
# else:
#     print("해당 열에 NaN 값이 없습니다.")

In [33]:
# # 행, 열 이름 생성
# new_index = [str(i+1) for i in range(user_place.shape[0])]
# user_place.index = new_index

# new_columns = [str(i+1) for i in range(user_place.shape[1])]
# user_place.columns = new_columns

# # 변경된 데이터프레임 저장
# user_place.to_csv('updated_user_place.csv', index=True, encoding='utf-8-sig') 

In [42]:
# 행, 열 이름 생성하느라 1차원씩 증가함
print(user_place.shape)
print(user_product.shape)
print(user_video.shape)

(31176, 4697)
(31176, 5834)
(31177, 3250)


In [43]:
# 유저가 장소에 대한 평가 유무
# 0의 개수
# zero_count = (user_place == 0).sum(axis=1)
# total_zero_count = zero_count.sum()

# 0이 아닌 숫자의 개수
non_zero_count = (user_place != 0).sum(axis=1)
# total_non_zero_count = non_zero_count.sum()

# 장소 평가 유무
count_zero = (non_zero_count == 0).sum()
count_non_zero = (non_zero_count != 0).sum()

# print(f"0의 개수: {zero_count}")
# print(f"0의 총 개수: {total_zero_count}")
print(f"0이 아닌 숫자의 개수: {non_zero_count}")
# print(f"0이 아닌 숫자의 총 개수: {total_non_zero_count}")
print(f"장소에 대한 평가가 없는 유저 수: {count_zero}")
print(f"장소에 대한 평가가 있는 유저 수: {count_non_zero}")

0이 아닌 숫자의 개수: 0         0
1         0
2         1
3         0
4        16
         ..
31171     0
31172     0
31173     0
31174     0
31175     0
Length: 31176, dtype: int64
장소에 대한 평가가 없는 유저 수: 8756
장소에 대한 평가가 있는 유저 수: 22420


In [44]:
# 유저가 상품에 대한 평가 유무
# 0의 개수
# zero_count = (user_product == 0).sum(axis=1)
# total_zero_count = zero_count.sum()

# 0이 아닌 숫자의 개수
non_zero_count = (user_product != 0).sum(axis=1)
# total_non_zero_count = non_zero_count.sum()

# 상품 평가 유무
count_zero = (non_zero_count == 0).sum()
count_non_zero = (non_zero_count != 0).sum()

# print(f"0의 개수: {zero_count}")
# print(f"0의 총 개수: {total_zero_count}")
print(f"0이 아닌 숫자의 개수: {non_zero_count}")
# print(f"0이 아닌 숫자의 총 개수: {total_non_zero_count}")
print(f"상품에 대한 평가가 없는 유저 수: {count_zero}")
print(f"상품에 대한 평가가 있는 유저 수: {count_non_zero}")

0이 아닌 숫자의 개수: 0        0
1        0
2        0
3        0
4        1
        ..
31171    4
31172    0
31173    0
31174    0
31175    1
Length: 31176, dtype: int64
상품에 대한 평가가 없는 유저 수: 28180
상품에 대한 평가가 있는 유저 수: 2996


In [45]:
# 유저가 영상에 대한 평가 유무
# 0의 개수
# zero_count = (user_video == 0).sum(axis=1)
# total_zero_count = zero_count.sum()

# 0이 아닌 숫자의 개수
non_zero_count = (user_video != 0).sum(axis=1)
# total_non_zero_count = non_zero_count.sum()

# 영상 평가 유무
count_zero = (non_zero_count == 0).sum()
count_non_zero = (non_zero_count != 0).sum()

# print(f"0의 개수: {zero_count}")
# print(f"0의 총 개수: {total_zero_count}")
print(f"0이 아닌 숫자의 개수: {non_zero_count}")
# print(f"0이 아닌 숫자의 총 개수: {total_non_zero_count}")
print(f"영상에 대한 평가가 없는 유저 수: {count_zero}")
print(f"영상에 대한 평가가 있는 유저 수: {count_non_zero}")

0이 아닌 숫자의 개수: 0         0
1         3
2         4
3         0
4        23
         ..
31172     1
31173     0
31174     0
31175     0
31176     0
Length: 31177, dtype: int64
영상에 대한 평가가 없는 유저 수: 20110
영상에 대한 평가가 있는 유저 수: 11067
